# Workbench API QA Example Sheet

This Jupyter notebook shows how to programatically connect to the QA data set using our APIs.

QA's data is indexed inside the same lake as our PCAP - Tick History Parquet data, and this notebook will also demonstrate that.

In [1]:
#
# This example shows selecting information the QA source.
#
 
from maystreet_data.api_client import ApiClient

query = """
SELECT
    *
FROM
    treactchg
WHERE
    activationdate >= TIMESTAMP '2022-01-01 00:00:00'
"""

with ApiClient() as client:
    df = client.qa.exec(sql=query, return_type="df")

df.head()

,estpermid,measure,isparent,pertype,perenddate,fyemonth,wintype,winlength,winenddate,winendoffset,...,defscale,normactchg,normcurrpermid,normscale,normsplitfactor,normcurradj,actchgpct,regionpermid,month,year
0,30064829281,9,False,3,2021-04-30,4,2,5,2021-06-15 02:00:00,-240,...,NaN,NaN,<NA>,NaN,1.0,1.0,NaN,110173,NaN,<NA>
1,30064877171,9,False,4,2021-10-31,10,2,5,2021-12-14 07:46:00,-300,...,NaN,NaN,<NA>,NaN,1.0,1.0,NaN,110173,NaN,<NA>
2,30064827025,9,False,3,2021-10-31,1,2,5,2021-11-23 16:01:00,-300,...,NaN,NaN,<NA>,NaN,1.0,1.0,NaN,110173,NaN,<NA>
3,30064878013,9,False,4,2021-10-31,10,2,5,2022-02-08 03:01:00,-300,...,NaN,NaN,<NA>,NaN,1.0,1.0,NaN,110173,NaN,<NA>
4,30064879634,9,False,4,2021-10-31,10,2,5,2022-01-19 21:14:00,-300,...,NaN,NaN,<NA>,NaN,1.0,1.0,NaN,110173,NaN,<NA>


In [6]:
#
# This example shows selecting information from both our PCAP - Tick History source and our QA source.
#

from maystreet_data.api_client import ApiClient

query = """
SELECT
    avg_aapl_price.dt,
    avg_aapl_price.price,
    treactchg.estpermid,
    treactchg.actgrowth
FROM 
    treactchg
        JOIN
    (
        SELECT 
            dt,
            AVG(price) AS price 
        FROM
            mt_trade 
        WHERE 
            f = 'bats_edga' 
            AND product = 'AAPL' 
            AND dt = '2023-06-07'
        GROUP BY
            dt
    ) AS avg_aapl_price ON CAST(CAST(activationdate AS DATE) AS VARCHAR) = avg_aapl_price.dt 
"""

with ApiClient() as client:
    df = client.foundation.exec(sql=query, return_type="df")

df.head()

,dt,price,estpermid,actgrowth
0,2023-06-07,178.805528,30064836358,0.14644
